In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
import hazm

In [4]:

def create_output(output_type, marker, span):
    return {"type":output_type, "marker":marker, "span":repr(span)}

In [5]:
import re
import json


class Symbolizer:
    def __init__(self):
        self.symbols = []
        with open('symbols.json') as symbols_file:
            data = json.load(symbols_file)
            for symbol in data['symbols']:
                self.symbols.append(symbol)
                # TODO: we can add more information to a symbol here
                # namad_description.append(n[1])

    def symbolize(self, sentence):
        output = []
        for namad in self.symbols:
            for m in re.finditer(rf'\b{namad}\b', sentence):
                output.append(create_output("نماد", namad, m.span()))
        return output

In [6]:
events = [
    "سود",
    "ضرر",
    "مثبت",
    "منفی",
    "صعود",
    "نزول",
    "افزایش",
    "کاهش",
    "رشد",
    "ریزش",
    "افشا",
    "اصلاح",
    "افت",
    "نوسان",
    "نوسانات",
    "سقوط",
    "تقسیم سود",
    "تعیین نرخ",
    "عرضه اولیه",
    "صف خرید",
    "صف فروش",
    "خروج سرمایه",
    "افزایش سرمایه"
    "خروج پول",
]
official_expression = [
    "اطلاعیه",
    "مجمع عمومی",
    "گزارش",
]
stock_expression = [
    "سهم رانتی",
    "تحلیل تکنیکال",
    "تحلیل فاندامنتال",
    "کندل استیک",
    "سیگنال",
    "تکنیکال",
    "کندل",
    "تیک",
    "سرشانه",
    "مقاومت",
    "حمایت",
    "کراس",
    "واگرایی",
    "اندیکاتور",
]
stock_characters =[
    "بازیگر",
    "حقیقی",
    "حقوقی",
    "بازی‌گردان",
    "نوسان‌گیر",
]
reason_propositions = [
    "دلیل",
    "باعث",
    "موجب",
    "برای"
]


In [7]:
def event_extractor(noun_phrase, phrase):
    output=[]
    for event in events:
        for m in re.finditer(event, noun_phrase.group()):
            body = noun_phrase.group()[1:-1-len(phrase)].strip()
            for reason_proposition in reason_propositions:
                if body.startswith(reason_proposition):
                    body = body[len(reason_proposition):]
            output.append(create_output("واقعه", body, noun_phrase.span()))
                
    return output


In [8]:
def get_phrase(tagged_str, phrase):
    # for b in  re.finditer(r"\[[آ-یء چ]+NP\]", a):
    phrases = []
    for b in re.finditer(rf"\[[^\]]*{phrase}\]", tagged_str):
        phrases.append(b.group())
        extracted_event = event_extractor(b, phrase)
        if extracted_event:
            print(extracted_event)
    return phrases


In [9]:
wrong_sentences = [
    "جریان اغاز معماملات فزر با ۱۵.۲ واحد تاثیر مثبت بر روند صعودی بازار فرابورس اثرگذار بود.", # فزر توی نمادامون نیست
    "کارشناسان روند متعادل و گاها نزولی برای بازار سهام امروز پیش‌بینی کرده‌اند", # اگر بعد از کارشناسان ، بزاریم درست تشخیص میده  وگرنه ابهام داره واقعا
    "بورس سه‌شنبه افت بیش از ۳۳ هزار واحدی داشت و در محدوده یک میلیون و ۴۴۶ هزار واحدی ایستاد.", # افت بیش از... رو تشخیص نمیده
    "امروز چند سیگنال برای سهامداران وجود دارد.", # چند
    "این تعیین نرخ به صورت رسمی، می‌تواند تاثیر مثبتی روی سوددهی شرکت‌ها داشته باشد", #این
    "کارشناسان معتقدند که با توافق، بورس در میان مدت و بلند مدت روند صعودی به خود می‌گیرد.", # بازم کاما
    "شرکت پالایش نفت شیراز با نماد شراز در مجمع عمومی فوق العاده ۱۲ ماهه افزایش سرمایه خود اعلام کرد", # بازم کاما

]
correct_sentences = [
    "ارزش سهام شرکت نفت کاهش یافت",
    "قرارداد با آمریکا باعث افت قیمت سهم وغدیر شد",
    "ریزش بازار به دلیل حمله‌ی روسیه است.",
    "فک کنم یه اصلاح قیمتی و کمی ریزش داشته باشیم.",
    "یک نکته‌ی تکنیکالی هم در صورت دستکاری نشدن اضافه کنم، کندلی که روز سه شنبه‌ی گذشته ثبت کرد کامل است",
    "روز چهارشنبه یه دفعه برای خودشون افشا زدن",
    "رشد قیمت‌ها باعث ایجاد صف خرید در سهم پرشیا شد",
    "شاخص به ۲ میلیون می‌رسه",
    "آمریکا باعث ریزش بازار شد",
    "آمریکا موجب ریزش بازار شد",
    "آمریکا دلیل ریزش بازار شد",
    "کاهش قیمت سهم عجیب بود",
    "قیمت زیاد شد",
    "قیمت زیاد است",
    "سهم قیمتش پایین است",
    "حضور تو موجب خوشحالی من در هوای بارانی است",
    "برکت همین افشای ب باعث شد سهم سه درصد مثبت شود",
    "بخاطر همین میگم پیگیر باشید",
    "گزارش فعالیت ماهانه دوره ۱ ماهه منتهی به ۱۴۰۰/۰۹/۳۰ برای دیران منتشر شد",
    "نرخ گاز خوراک پتروشیمی‌ها طبق فرمول ابلاغی وزارت نفت و با اعمال سقف نرخ گاز صادراتی، حداکثر پنج هزار تومان تعیین شده است",
    "نرخ گاز مصرفی صنایع سیمان و سایر صنایع معادن ۱۰ درصد نرخ خوراک پتروشیمی‌ها تعیین شده است",
    "کارشناسان می‌گویند تعیین شدن نرخ خوراک پتروشیمی‌ها به سودآوری شرکت‌ها در بلند مدت کمک می‌کند",
    "قیمت دلار بازار آزاد به بیش از ۲۸ هزار تومان رسید. کارشناسان عقیده دارند افزایش مقطعی قیمت دلار، تاثیر منفی روی بازار سرمایه دارد",
    "عرضه اولیه جدید می‌تواند، نقدینگی بیشتری به بازارا سرمایه وارد کند","کارشناسان می‌گویند که اگر ارز ۴۲۰۰ تومانی حذف شود، صنعت دارو در کوتاه مدت نزولی می‌شود",
    "شرکت سرمایه‌گذاری توسعه صنعت و تجارت اعلام کرد که در نظر دارد نسبت به فروش ملک خود واقع در شهر کرمان از طریق مزایده عمومی اقدام کند",
    "کارشناسان می‌گویند کاهش نرخ سود بین بانکی می‌تواند نقدینگی جدید به بورس وارد کند.",
    "کارشناسان می‌گویند که کاهش تامین مالی از بورس می‌تواند این بازار را صعودی و نقدینگی جدید وارد آن کند.",
    "جلسه معارفه آسیاتک روز دوشنبه برگزار می‌شود. (سنا) احتمالا آسیاتک اولین عرضه اولیه بازار سرمایه در قرن جدید باشد.",
    "پذیره‌نویسی نخستین صندوق سهامی با نماد سلام از روز دوشنه در بازار سهام آغاز می‌شود.",
    "کارشناسان معتقدند که پذیره‌نویسی صندوق‌های جدید نقدینگی جریان نقدینگی در بازار را تقویت می‌کند",
    "شرکت پالایش نفت اصفهان با نماد شپنا افزایش سرمایه خود را از محل سود انباشته ثبت کرد.",
    "ولبهمن قرار است سرمایه ۸۰‌ میلیارد تومانی خود را به ۱۰۰ میلیارد تومان برسد.",
]

In [10]:
new_sentences = wrong_sentences+correct_sentences

In [11]:
tagger = hazm.POSTagger(model="../resources/postagger.model")
chunker = hazm.Chunker(model="../resources/chunker.model")
for sentence in new_sentences:
    # print("-------------")
    # print(sentence)
    # print("symbols: ")
    symbolizer = Symbolizer()
    output = symbolizer.symbolize(sentence)
    if output:
        print(output)

    tagged = tagger.tag(hazm.word_tokenize(sentence))
    # print("tagged: ")
    # print(tagged)
    chunked_str = hazm.tree2brackets(chunker.parse(tagged))
    # print("chukned: ")
    # print(chunked_str)
    get_phrase(chunked_str, phrase="NP")
    get_phrase(chunked_str, phrase="ADJP")


[{'type': 'نماد', 'marker': 'فرابورس', 'span': '(68, 75)'}]
[{'type': 'واقعه', 'marker': '۱۵.۲ واحد تاثیر مثبت', 'span': '(37, 62)'}]
[{'type': 'واقعه', 'marker': 'روند صعودی بازار فرابورس', 'span': '(71, 100)'}]
[{'type': 'واقعه', 'marker': 'کارشناسان روند متعادل و گاها نزولی', 'span': '(0, 39)'}]
[{'type': 'نماد', 'marker': 'بورس', 'span': '(0, 4)'}]
[{'type': 'واقعه', 'marker': 'افت بیش', 'span': '(18, 30)'}]
[{'type': 'واقعه', 'marker': 'چند سیگنال', 'span': '(13, 28)'}]
[{'type': 'واقعه', 'marker': 'این تعیین نرخ', 'span': '(0, 18)'}]
[{'type': 'واقعه', 'marker': 'تاثیر مثبتی', 'span': '(58, 74)'}]
[{'type': 'واقعه', 'marker': 'سوددهی شرکت\u200cها', 'span': '(84, 103)'}]
[{'type': 'نماد', 'marker': 'بورس', 'span': '(31, 35)'}]
[{'type': 'واقعه', 'marker': 'میان مدت و بلند مدت روند صعودی', 'span': '(70, 105)'}]
[{'type': 'نماد', 'marker': 'شراز', 'span': '(30, 34)'}, {'type': 'نماد', 'marker': 'شیراز', 'span': '(16, 21)'}, {'type': 'نماد', 'marker': 'پالایش', 'span': '(5, 11)'}]
[{

In [9]:
import hazm
from __future__ import unicode_literals
tagger = hazm.POSTagger(model='../resources/postagger.model')
print(tagger.tag(hazm.word_tokenize('ما بسیار کتاب می‌خوانیم'))[3][0])
# tagger.tag(hazm.word_tokenize('ما بسیار کتاب می‌خوانیم'))
tagger.tag(hazm.word_tokenize('کتاب خواندن را دوست داریم'))
tagger.tag(hazm.word_tokenize('من درخت بلند را دیدم'))
tagger.tag(hazm.word_tokenize('مرد زبان نفهم'))
tagger.tag(hazm.word_tokenize('مرد سود ده'))

می‌خوانیم


[('مرد', 'Ne'), ('سود', 'Ne'), ('ده', 'NUM')]

In [10]:
word = 'کتاب خواندن را دوست داریم'
word = '۵ گل زیبا خریدم'
word = 'سهام ۳ واحد افزایش یافت'
word = 'هوا ۵ درجه گرم شد'
chunker = hazm.Chunker(model='../resources/chunker.model')
chunker = hazm.RuleBasedChunker()
tagged = tagger.tag(hazm.word_tokenize(word))
print(chunker.parse(tagged))
print(hazm.tree2brackets(chunker.parse(tagged)))
chunker.parse(tagged).draw()

(S (NP هوا/N) (NP ۵/NUM درجه/N) (ADJP گرم/AJ) (VP شد/V))
[هوا NP] [۵ درجه NP] [گرم ADJP] [شد VP]


In [14]:
import nltk

nltk.ChunkParserI('a big cat was dead')

TypeError: ChunkParserI() takes no arguments

In [ ]:
lst = [u'\u200c', u'\u5de5', 'ب\u200cر', 'امی‌ن']
print(lst)
print('امی\u200cن')
print('امی‌ن')

['\u200c', '工', 'ب\u200cر', 'امی\u200cن']
امی‌ن
امی‌ن


In [13]:
import nltk
nltk.download('tagsets')
nltk.help.upenn_tagset()

[nltk_data] Downloading package tagsets to /home/fatemeh/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw a

In [12]:
# nltk.download('punkt')
# nltk.download('popular')
# nltk.download('universal_tagset')
# text = nltk.word_tokenize("He is a good, tall boy")
text = nltk.word_tokenize("He is a little tall boy")
# text = nltk.word_tokenize("the little crazy cat sat on the mat")
# text = nltk.word_tokenize("amin is clever and good boy")
# text = nltk.word_tokenize("And now for something completely different")
nltk.pos_tag(text, tagset='universal')
# nltk.pos_tag(text)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  Searched in:
    - '/home/fatemeh/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/home/fatemeh/uni/Term8/NLP/nlp_env/nltk_data'
    - '/home/fatemeh/uni/Term8/NLP/nlp_env/share/nltk_data'
    - '/home/fatemeh/uni/Term8/NLP/nlp_env/lib/nltk_data'
    - ''
**********************************************************************


In [ ]:
unchunked_sent = 'the little cat sat on the mat'
rule1 = nltk.chunk.regexp.ChunkRule('<NN|DT>+',
    'Chunk sequences of NN and DT')
chunkparser = nltk.chunk.regexp.RegexpParser( [rule1] )
chunkparser.parse(unchunked_sent)

TypeError: Expected string or list of RegexpChunkParsers for the grammar.

In [ ]:
import nltk
text = "learn php from guru99"
text = 'the little crazy cat is sitting on the mat'
tokens = nltk.word_tokenize(text)
print(tokens)
tag = nltk.pos_tag(tokens)
print(tag)
grammar = """
NP: {<DT>?<JJ>*<NN>} 
VP: {<VB.?>*}
"""
cp  = nltk.RegexpParser(grammar)
result = cp.parse(tag)
print(result)
result.draw()    # It will draw the pattern graphically which can be seen in Noun Phrase chunking 

['the', 'little', 'crazy', 'cat', 'is', 'sitting', 'on', 'the', 'mat']
[('the', 'DT'), ('little', 'JJ'), ('crazy', 'JJ'), ('cat', 'NN'), ('is', 'VBZ'), ('sitting', 'VBG'), ('on', 'IN'), ('the', 'DT'), ('mat', 'NN')]
(S
  (NP the/DT little/JJ crazy/JJ cat/NN)
  (VP is/VBZ sitting/VBG)
  on/IN
  (NP the/DT mat/NN))


In [1]:
# text = 'زنگ‌ها برای که به صدا درمی‌آید؟' 
# text = ' ۵ دختر خوب مدرسه‌ی ما گل'
# text = 'گل باغ شهر ما'
# text = '۳ سگ وحشی زشت علی زیبا به دریا رفتند.'
# text = 'گل زیبای خانه‌ی مادربزرگ من'
text = 'سهام ۳ واحد افزایش یافت'
tagger = hazm.POSTagger(model='../resources/postagger.model')
lemmatizer = hazm.Lemmatizer()
parser = hazm.DependencyParser (tagger = tagger, lemmatizer = lemmatizer, working_dir='../resources')
# print(parser.parse(hazm.word_tokenize(text)).tree().pprint())
parser.parse(hazm.word_tokenize(text))

NameError: name 'hazm' is not defined

استفاده از FIS روی توییت ها 
افعال مثبت و منفی و کلمات و متفی در تنفی مثبت و اینا
فراکاپ
فرق گشتن دنبال keywordها در NP و VP
یه کلمه‌ی مثل افت توی ADJP هم مهمه باید بررسی کنیم.